In [1]:
from __future__ import print_function
import cv2
import argparse
import time
import numpy as np
import time
from vidstab import VidStab
from PIL import Image
from skimage.measure import label, regionprops
import os

from CentroidTracker import CentroidTracker, progressBar, showWindows, georeference
import params
from collections import defaultdict
from math import sqrt
import os
import matplotlib.pyplot as plt

# ---- import DeepLab
from model import DeepLabModel, DeepLab

DeepLab Module

In [ ]:
# ---- input video file path
video = str('./input/DJI_0008_original.MP4')

# ---- output video file path
output_video = str('./output/DJI_0008_short_DL.avi') # Path to Output-Video

# ---- output stabilized video path
video_stabilized = str('./repo/output_stab.avi') # Path to stabilized Video

## Stabilize Video

In [ ]:
# Stabilize Video and save to File "video_stabilized"
stabilizer = VidStab(kp_method='FAST', threshold=42, nonmaxSuppression=False)
stabilizer.stabilize(input_path = video, output_path = video_stabilized, border_type = 'black', border_size=100)

stabilizer.plot_trajectory()
plt.show()

stabilizer.plot_transforms()
plt.show()


## DeepLab Module

In [ ]:
# ---- deeplab definitions

def create__label_colormap():
     return np.asarray([
         [0, 0, 0],
         [0, 192, 0],
         [255, 192, 192],
     ])
    

def label_to_color_image(label):
    """Adds color defined by the dataset colormap to the label.

    Args:
    label: A 2D array with integer type, storing the segmentation label.

    Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

    Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
    """
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    colormap = create__label_colormap()

    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')

    return colormap[label]

In [ ]:
current_frame = 0

# input DeepLab
input_DL = './repo/output_short_stab.avi'

In [ ]:
# ---- iterate DeepLab trough images

model = DeepLabModel('model/frozen_inference_graph_cars.pb')

cap = cv2.VideoCapture(input_DL)
cap.set(cv2.CAP_PROP_POS_AVI_RATIO,0)

img_array = []

while True:
    current_frame += 1
    ret, frame = cap.read()
    
    if frame is None:
        print('finished!')
        break
        
    mask = model.run(Image.fromarray(frame))
    
    label_img = label(mask)
    img_array.append(label_to_color_image(mask).astype(np.uint8))
    
    
# ---- translate array
size = img_array[0].shape[:2][::-1]

# ---- save to video
out = cv2.VideoWriter(output_video,cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

## Optical Flow Module

In [ ]:
path = defaultdict(list)

In [ ]:
# Set basic variables
ct = CentroidTracker()
current_frame = 0
id = 1

In [ ]:
backSub = cv2.createBackgroundSubtractorMOG2()

In [ ]:
#Georeferencing
coord_pixel0_x = 0
coord_pixel0_y = 0
scale = 1

In [ ]:
# Start time
start = time.time()

In [ ]:
capture = cv2.VideoCapture(output_video)

# running the loop
# extracting the frames
ret, first_frame = capture.read()

# converting to gray-scale
gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

while True:
    current_frame += 1
    ret, frame = capture.read()

#     if frame is None:
#         print('finished!')
#         break
    
    
    fgMask = backSub.apply(frame)
    # optimization of elements
    thresh = cv2.threshold(fgMask, params.threshold, 255, cv2.THRESH_BINARY)[1]
    se0 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, se0)
    kernel_dil = np.ones(params.kernel_dilatation, np.uint8)
    thresh = cv2.dilate(thresh, kernel_dil, iterations=1)

    se1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
    se2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (41, 41))
    opened = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, se1)
    closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, se2)
    
    # detect the contours on the binary image using cv2.CHAIN_APPROX_NONE
    contours, hierarchy = cv2.findContours(image=closed, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
    mask = np.zeros_like(first_frame)
    # draw contours on the original image
    contours = [x for x in contours if len(x) >= 30]
    cv2.drawContours(image=mask, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2,
                     lineType=cv2.LINE_AA)
    
    rects = []
    ROI_number = 0
    for cntr in contours:
        x, y, w, h = cv2.boundingRect(cntr)
        if params.min_width < w < params.max_width and params.min_height < h < params.max_height:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            ROI = mask[y:y + h, x:x + w]
            cv2.putText(mask, str(ROI_number), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))
            rects.append([x, y, x + w, y + h])
            ROI_number += 1
    
    
    # update centroid tracker using the computed set of bounding box rectangles
    objects, life = ct.update(rects)
    
    #build path in swiss coordinate system
    if objects is not None:
        for (objectID, centroid) in objects.items():
            if life[objectID] > params.min_life:
                # draw both the ID of the object and the centroid of the object on the output frame
                text = "ID {}".format(objectID)
                x = centroid[0]
                y = centroid[1]
                cv2.putText(frame, text, (x - 20, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                cv2.circle(frame, (x, y), 4, (255, 255, 255), -1)
                if objectID in path.keys():
                    path[objectID].append((coord_pixel0_x+(x*scale), coord_pixel0_y-(y*scale)))
                else:
                    path[objectID] = [(coord_pixel0_x+(x*scale), coord_pixel0_y-(y*scale))]
                    
    # End time
    end = time.time()
    # Calculate frames per second
    fps = round(current_frame / (end-start), 2)
    
    
    
    imS = cv2.resize(mask, (960, 540))                    # Resize image
    cv2.imshow("output", imS) 
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

In [ ]:
#############----------- RUN ALL ABOVE ---------------